In [952]:
import numpy as np
import math
import random
from functools import reduce

In [953]:
def generateRandomVector(d):
    return np.random.randint(2, size = d)

In [954]:
dimensions = 10000

A = generateRandomVector(dimensions)
B = generateRandomVector(dimensions)

In [955]:
def similarity(A, B):
    if A.size != B.size:
        print('A and B have different dimensions.')
    
    count = 0
    for i in range(0,(A.size - 1)):
        if A[i]==B[i]:
            count += 1
    
    return count/A.size


In [956]:
def calculateSTDs(A, B):
    return abs((similarity(A,B) * A.size  - 5000) / 50)

In [957]:
def applyThreshold(V, n):
    if n % 2 == 1:
        threshold = lambda x: 1 if x > n/2 else 0
    else:
        threshold = lambda x: 1 if x > n/2 else (1 if x == n/2 and bool(random.getrandbits(1)) else 0)

    vectorized_threshold = np.vectorize(threshold)
    return vectorized_threshold(V)

In [958]:
def arithmeticSum(A, B):
    sum = A + B

    return applyThreshold(sum, 2) 

In [959]:
def arithmeticSubtraction(A, B):
    compliment = 1 - B

    return arithmeticSum(A, compliment)

In [971]:
def arithmeticSumVectors(vectors):
    sum = reduce(lambda x, y: x+y, vectors)

    return applyThreshold(sum, len(vectors))

In [961]:
mean = arithmeticSum(A, B)

In [962]:
def comparison(A, B):
    dividend = sum(A*B)
    divisor = math.sqrt(sum(A**2) * sum(B**2))

    return dividend / divisor

In [963]:
print('sim  A, A:         ', similarity(A,A))
print('sim  A, B:         ', similarity(A,B))
print('sim  A, mean:      ', similarity(A,mean))
print('sim  B, mean:      ', similarity(B,mean))

print('comp A, B:         ', comparison(A, B))
print('comp A, mean:      ', comparison(A, mean))
print('comp B, mean:      ', comparison(B, mean))

print('STDs A, B:         ', calculateSTDs(A, B))
print('STDs A, mean:     ', calculateSTDs(A, mean))
print('STDs B, mean:     ', calculateSTDs(B, mean))

print('sim  A, mean - B:  ',similarity(A, arithmeticSubtraction(mean, B)))
print('STDs A, mean - B: ',calculateSTDs(A, arithmeticSubtraction(mean, B)))

print('sim  B, mean - B:  ',similarity(B, arithmeticSubtraction(mean, B)))
print('STDs B, mean - B: ',calculateSTDs(B, arithmeticSubtraction(mean, B)))

sim  A, A:          0.9999
sim  A, B:          0.4985
sim  A, mean:       0.7545
sim  B, mean:       0.7439
comp A, B:          0.5065673784975031
comp A, mean:       0.7585714355202725
comp B, mean:       0.7506332149763301
STDs A, B:          0.3
STDs A, mean:      50.899999999999984
STDs B, mean:      48.78
sim  A, mean - B:   0.6335
STDs A, mean - B:  25.58
sim  B, mean - B:   0.3759
STDs B, mean - B:  26.25999999999999


In [964]:
def mult(A, B):
    return np.bitwise_xor(A, B)

In [965]:
print(A)
print(B)
print(mult(A, B))
print(comparison(A, mult(mult(A, B), B)))

[1 1 0 ... 1 0 0]
[1 0 0 ... 1 0 1]
[0 1 0 ... 0 0 1]
1.0


In [966]:
class ItemMemory:
    def __init__(self, vectors = []):
        self.vectors = vectors
    
    def addVector(self, label, V):
        self.vectors.append((label, V))

    def cleanUp(self, V):
        return max(self.vectors, key=lambda x: comparison(V, x[1]))


In [967]:
itemMemory = ItemMemory()
itemMemory.addVector('A', A)
itemMemory.addVector('B', B)
itemMemory.addVector('mean', mean)

In [968]:
print(itemMemory.vectors)
print(itemMemory.cleanUp(B))

[('A', array([1, 1, 0, ..., 1, 0, 0])), ('B', array([1, 0, 0, ..., 1, 0, 1])), ('mean', array([1, 1, 0, ..., 1, 0, 1]))]
('B', array([1, 0, 0, ..., 1, 0, 1]))


In [978]:
itemMemoryA = ItemMemory()

X = generateRandomVector(dimensions)
A = generateRandomVector(dimensions)

Y = generateRandomVector(dimensions)
B = generateRandomVector(dimensions)

Z = generateRandomVector(dimensions)
C = generateRandomVector(dimensions)

itemMemoryA.addVector('A', A)
itemMemoryA.addVector('B', B)
itemMemoryA.addVector('C', C)
itemMemoryA.addVector('X', X)
itemMemoryA.addVector('Y', Y)
itemMemoryA.addVector('Z', Z)

H = arithmeticSumVectors([mult(A, X), mult(B, Y), mult(C, Z)])

unbinded = itemMemory.cleanUp(mult(H, X))

print(unbinded)

print(comparison(unbinded, A))

('A', array([0, 0, 0, ..., 0, 0, 0]))


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.